Aim: Demonstrate the working model and principle of candidate elimination algorithm. 

Program: For a given set of training data examples stored in a .CSV file, implement and demonstrate the Candidate-Elimination algorithm to output a description of the set of all hypotheses consistent with the training examples.

In [2]:
import numpy as np 
import pandas as pd

In [5]:
data = pd.read_csv('training_data.csv', header=None) 
concepts = np.array(data.iloc[:,:-1])

In [6]:
print("\nInstances are:\n",concepts)
target = np.array(data.iloc[:,-1])
print("\nTarget Values are: ",target)


Instances are:
 [['Sunny' ' Warm' ' Normal' ' Strong' ' Warm' ' Same']
 ['Sunny' ' Warm' ' High' ' Strong' ' Warm' ' Same']
 ['Rainy' ' Cold' ' High' ' Strong' ' Warm' ' Change']
 ['Sunny' ' Warm' ' High' ' Strong' ' Cool' ' Change']]

Target Values are:  [' Yes' ' Yes' ' No' ' Yes']


In [8]:
def learn(concepts, target):
    specific_h = concepts[0].copy()
    print("\nInitialization of specific_h and genearal_h")
    print("\nSpecific Boundary: ", specific_h)
    general_h = [["?" for i in range(len(specific_h))] for i in range(len(specific_h))] 
    print("\nGeneric Boundary: ",general_h)
    for i, h in enumerate(concepts): 
        print("\nInstance", i+1 , "is ", h) 
        if target[i] == "Yes":
            print("Instance is Positive ") 
            for x in range(len(specific_h)):
                if h[x]!= specific_h[x]: 
                    specific_h[x] ='?' 
                    general_h[x][x] ='?'
        if target[i] == "No": 
            print("Instance is Negative ") 
            for x in range(len(specific_h)):
                if h[x]!= specific_h[x]: 
                    general_h[x][x] = specific_h[x]
                else:
                    general_h[x][x] = '?'
        print("Specific Bundary after ", i+1, "Instance is ", specific_h) 
        print("Generic Boundary after ", i+1, "Instance is ", general_h)

        print("\n")
    indices = [i for i, val in enumerate(general_h) if val == ['?', '?', '?', '?', '?', '?']] 
    for i in indices:
        general_h.remove(['?', '?', '?', '?', '?', '?']) 
    return specific_h, general_h
s_final, g_final = learn(concepts, target)
print("Final Specific_h: ", s_final, sep="\n") 
print("Final General_h: ", g_final, sep="\n")


Initialization of specific_h and genearal_h

Specific Boundary:  ['Sunny' ' Warm' ' Normal' ' Strong' ' Warm' ' Same']

Generic Boundary:  [['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?']]

Instance 1 is  ['Sunny' ' Warm' ' Normal' ' Strong' ' Warm' ' Same']
Specific Bundary after  1 Instance is  ['Sunny' ' Warm' ' Normal' ' Strong' ' Warm' ' Same']
Generic Boundary after  1 Instance is  [['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?']]



Instance 2 is  ['Sunny' ' Warm' ' High' ' Strong' ' Warm' ' Same']
Specific Bundary after  2 Instance is  ['Sunny' ' Warm' ' Normal' ' Strong' ' Warm' ' Same']
Generic Boundary after  2 Instance is  [['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?',

In [10]:
def learn(concepts, target):
    num_attributes = len(concepts[0])
    specific_h = concepts[0].copy()  # Initialize the specific hypothesis
    print("\nInitialization of Specific_h and General_h:")
    print("Specific Boundary:", specific_h)

    # Initialize General Hypotheses (one most general hypothesis)
    general_h = [['?' for _ in range(num_attributes)]]
    print("Generic Boundary:", general_h)

    for i, h in enumerate(concepts): 
        print(f"\nInstance {i+1} is {h}") 
        if target[i] == "Yes":
            print("Instance is Positive") 
            specific_h = update_specific(specific_h, h)
            general_h = update_general(general_h, specific_h)
        elif target[i] == "No": 
            print("Instance is Negative") 
            general_h = [g for g in general_h if not is_more_specific(specific_h, g)]

        print("Specific Boundary after instance", i+1, "is", specific_h) 
        print("Generic Boundary after instance", i+1, "is", general_h)
        print("\n")

    # Remove redundant general hypotheses
    final_general_h = []
    for g in general_h:
        if not any(is_more_specific(g, g2) for g2 in final_general_h):
            final_general_h.append(g)
    
    return specific_h, final_general_h

def update_specific(specific, example):
    """Update the Specific set based on the positive example."""
    for i in range(len(specific)):
        if specific[i] == '0' or specific[i] == example[i]:
            specific[i] = example[i]
        else:
            specific[i] = '?'
    return specific

def update_general(general, specific):
    """Update the General set based on the positive example."""
    new_general = []
    for g in general:
        if is_more_general(g, specific):
            new_general.append(g)
    
    # Generate new hypotheses by generalizing specific hypothesis
    specific_copy = specific[:]
    for i in range(len(specific)):
        if specific[i] == '?':
            continue
        temp_general = specific_copy[:]
        temp_general[i] = '?'
        if temp_general not in new_general:
            new_general.append(temp_general)
    
    return new_general

def is_more_general(g, s):
    """Check if a hypothesis g is more general than hypothesis s."""
    return all(g_i == '?' or g_i == s_i for g_i, s_i in zip(g, s))

def is_more_specific(s1, s2):
    """Check if hypothesis s1 is more specific than hypothesis s2."""
    return all(s1_i == s2_i or s2_i == '?' for s1_i, s2_i in zip(s1, s2))

# Example usage
concepts = [['sunny', 'warm', 'normal', 'strong', 'warm', 'same'],
            ['sunny', 'warm', 'high', 'strong', 'warm', 'same'],
            ['rainy', 'cold', 'high', 'strong', 'warm', 'change'],
            ['sunny', 'warm', 'high', 'strong', 'cool', 'change']]
target = ['Yes', 'Yes', 'No', 'Yes']

s_final, g_final = learn(concepts, target)
print("Final Specific_h:", s_final, sep="\n") 
print("Final General_h:", g_final, sep="\n")



Initialization of Specific_h and General_h:
Specific Boundary: ['sunny', 'warm', 'normal', 'strong', 'warm', 'same']
Generic Boundary: [['?', '?', '?', '?', '?', '?']]

Instance 1 is ['sunny', 'warm', 'normal', 'strong', 'warm', 'same']
Instance is Positive
Specific Boundary after instance 1 is ['sunny', 'warm', 'normal', 'strong', 'warm', 'same']
Generic Boundary after instance 1 is [['?', '?', '?', '?', '?', '?'], ['?', 'warm', 'normal', 'strong', 'warm', 'same'], ['sunny', '?', 'normal', 'strong', 'warm', 'same'], ['sunny', 'warm', '?', 'strong', 'warm', 'same'], ['sunny', 'warm', 'normal', '?', 'warm', 'same'], ['sunny', 'warm', 'normal', 'strong', '?', 'same'], ['sunny', 'warm', 'normal', 'strong', 'warm', '?']]



Instance 2 is ['sunny', 'warm', 'high', 'strong', 'warm', 'same']
Instance is Positive
Specific Boundary after instance 2 is ['sunny', 'warm', '?', 'strong', 'warm', 'same']
Generic Boundary after instance 2 is [['?', '?', '?', '?', '?', '?'], ['sunny', 'warm', '?', 's

In [ ]:


def learn(concepts, target):
specific_h = concepts[0].copy()
print("\nInitialization of specific_h and genearal_h")
print("\nSpecific Boundary: ", specific_h)
general_h = [["?" for i in range(len(specific_h))] for i in range(len(specific_h))] print("\nGeneric Boundary: ",general_h)
for i, h in enumerate(concepts): print("\nInstance", i+1 , "is ", h) if target[i] == "yes":
print("Instance is Positive ") for x in range(len(specific_h)):
if h[x]!= specific_h[x]: specific_h[x] ='?' general_h[x][x] ='?'
if target[i] == "no": print("Instance is Negative ") for x in range(len(specific_h)):
if h[x]!= specific_h[x]: general_h[x][x] = specific_h[x]
else:
general_h[x][x] = '?'
print("Specific Bundary after ", i+1, "Instance is ", specific_h) print("Generic Boundary after ", i+1, "Instance is ", general_h)
Dept. of AIML 2023-24 8
Machine Learning Laboratory 21AIL66
print("\n")
indices = [i for i, val in enumerate(general_h) if val == ['?', '?', '?', '?', '?', '?']] for i in indices:
general_h.remove(['?', '?', '?', '?', '?', '?']) return specific_h, general_h
s_final, g_final = learn(concepts, target)
print("Final Specific_h: ", s_final, sep="\n") print("Final General_h: ", g_final, sep="\n")